In [ ]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
#from losses import ssim_loss
#from mymetrics import ssim_metric,psnr_metric,ssim_loss
from keras.losses import mean_squared_error, mean_absolute_error
from skimage.metrics import structural_similarity as ssim
import skimage.metrics
import pytorch_ssim
import torch
from torch.autograd import Variable
from skimage import measure
import math
def ssim_metric2d(y_true, y_pred):
  return tf.reduce_mean(tf.image.ssim(y_true, y_pred, 2.0))

def ssim_metric3d(y_true, y_pred):
  return tf.reduce_mean(tf.image.ssim(y_true, y_pred, 2.0,filter_size=5))

def psnr_metric(y_true, y_pred):
    """
    PSNR is Peek Signal to Noise Ratio, which is similar to mean squared error.
    It can be calculated as
    PSNR = 20 * log10(MAXp) - 10 * log10(MSE)
    When providing an unscaled input, MAXp = 255. Therefore 20 * log10(255)== 48.1308036087.
    However, since we are scaling our input, MAXp = 1. Therefore 20 * log10(1) = 0.
    Thus we remove that component completely and only compute the remaining MSE component.
    """
    return -10. * K.log(K.mean(K.square(y_pred - y_true)))
def psnr_met1(y_true, y_pred):
    max_pixel = 1.0
    return (10.0 * K.log((max_pixel ** 2) / (K.mean(K.square(y_pred - y_true), axis=-1)))) / 2.303

# Loss functtion
def ssim_loss1(y_true, y_pred):
    myssim=tf.image.ssim(y_true, y_pred,1)
    s_3d_tf = tf.Session().run(myssim)
    return tf.reduce_mean((1.0-s_3d_tf)/2.0)

def ssim_loss2(y_true, y_pred):
    print(y_true.dtype)
    img1=torch.cuda.FloatTensor(y_true)
    img2=torch.cuda.FloatTensor(y_pred)
    #img1=tf.convert_to_tensor(y_true)
    #img2=tf.convert_to_tensor(y_pred)
    print(img1.dtype)
    print(img2.shape)
    ssim_3d_sk=pytorch_ssim.ssim3D(y_true, y_pred)
    #ssim_3d_sk = skimage.metrics.structural_similarity(y_true, y_pred, multichannel=False, gaussian_weights=True, data_range=1.0, use_sample_covariance=False)
    #ssim_3d_sk = ssim(img1, img2,win_size=3, gaussian_weights=True, data_range=1.0, use_sample_covariance=False)
    return tf.reduce_mean((1.0-ssim_3d_sk)/2.0)

# Loss functtion
def ssim_loss2d(y_true, y_pred):
    return tf.reduce_mean((1.0-tf.image.ssim(y_true, y_pred,1))/2.0)

def ssim_loss3d(y_true, y_pred):
    return tf.reduce_mean((1.0-tf.image.ssim_multiscale(y_true, y_pred,1,filter_size=7))/2.0)

def SSIMLoss(y_true, y_pred):
  return 1 - tf.reduce_mean(ssim(y_true, y_pred))

def psnr_met2(y_true, y_pred):
    max_pixel = 1.0
    return 10.0 * (1.0 / math.log(10)) * K.log((max_pixel ** 2) / (K.mean(K.square(y_pred - y_true))))

def L2_SSIM_loss(y_true, y_pred):
  alpha=0.5
  return alpha*mean_squared_error(y_true, y_pred) + (1.0-alpha)* ssim_loss(y_true, y_pred) 

def L1_SSIM_loss(y_true, y_pred):
  alpha=0.5
  return alpha*mean_absolute_error(y_true, y_pred) + (1.0-alpha)* ssim_loss(y_true, y_pred) 

def total_content_loss2d(y_true,y_pred):
  alpha=0.3
  tv_weight=1e-5
  '''
  Total variation loss is used to keep the image locally coherent
   '''
  #assert K.ndim(x) == 4
  a = tf.square(y_pred[:, :-1, :-1, :] - y_pred[:, 1:, :-1, :])
  b = tf.square(y_pred[:, :-1, :-1, :] - y_pred[:, :-1, 1:, :])
  #a = tf.square(x[:, : img_nrows - 1, : img_ncols - 1, :] - x[:, 1:, : img_ncols - 1, :])
  #b = tf.square(x[:, : img_nrows - 1, : img_ncols - 1, :] - x[:, : img_nrows - 1, 1:, :])
  return tv_weight*K.sum(K.pow(a + b,1.25))+alpha*mean_squared_error(y_true,y_pred)+(1-alpha)*mean_absolute_error(y_true,y_pred)

def L2_L1_loss(y_true,y_pred):
  alpha=0.4
  return alpha*mean_squared_error(y_true,y_pred)+(1-alpha)*mean_absolute_error(y_true,y_pred)

def mse_TV(y_true,y_pred):
  tv_weight=1e-5
  mse_weight=0.5
  return mse_weight*mean_squared_error(y_true,y_pred)+tv_weight*total_variation_loss(y_pred)


In [ ]:
custom_objects={'ssim_metric': ssim_metric,'psnr_met2':psnr_met2,'ssim_loss':ssim_loss}
model=load_model('/models/HSI/EnsRL3D-G10I20_040_MSE.h5',custom_objects=custom_objects)

In [ ]:
# Load test image
import numpy as np
from mymetrics import *
from ganModel import *
from mri_preprocess import *
from scipy.io import loadmat


from keras.models import load_model

#img3d=np.load('/data/HSI/CAVE/CAVE3d_06.npy')
#img3d=loadmat('/data/denoising/wdcP2.mat')['img'];
img3d=loadmat('/data/denoising/Salinas.mat')['salinas'];
#img3d=img3d[:,:,1:2]
print(img3d.shape)
#a=mri_3d_extract(img3d,patch_size=512,channels=31)
a=img3d.reshape(1,img3d.shape[0],img3d.shape[1],img3d.shape[2])
print(a.shape)

img_width,img_height,layers=a.shape[1],a.shape[2],a.shape[3]

#Reshaping data  
if K.image_data_format() == 'channels_first':
    print("channel first format")
    # Considering 3D data
    input_test = a.reshape(1, 1, img_width, img_height,layers)
    input_shape = (1, 1, img_width, img_height,layers)
else:
    print("channel last format")
    # Considering 3D data
    input_test = a.reshape(1, img_width, img_height,layers,1)
    input_shape = (1, img_width, img_height, layers, 1)
    
    # Considering 2D data
    #input_test = a.reshape(1, img_width, img_height,1)
    #input_shape = (1, img_width, img_height, 1)
    


#################### Normalize Data ##########################

# Normalise data between [0,1] (for each sample)
noisy_input_test = (input_test - np.min(input_test))/np.ptp(input_test)
# Normalise data between[0,255] (per sample) as integer: don't forget the parenthesis before astype(int)
#input_train[i] = (255*(input_train[i] - np.min(input_train[i]))/np.ptp(input_train[i])).astype(int)        
print(np.shape(input_test))  

#################### Add noise ##################
from numpy.random import seed
from numpy.random import randint
import sys
nb_path = '/content/drive/MyDrive/VAE/modules'
sys.path.append(nb_path)
from hsi_preprocess import *

# Adding noise for 3d data
#noisy_input_test=add_gaussian_noise3D(input_test,20)
#noisy_input_test=add_impulse_noise3D(noisy_input_test,5)

# normalize noisy data
#input_test=np.stack((noisy_input_test[:,:,:,30,0:1],noisy_input_test[:,:,:,24,0:1],noisy_input_test[:,:,:,20,0:1]),3)
#noisy_input_test = (noisy_input_test - np.min(noisy_input_test))/np.ptp(noisy_input_test)
# Adding noise for 2d data
#noisy_input_test=add_gaussian_noise(input_test,10)
#noisy_input_test=add_impulse_noise(noisy_input_test,10)
#noisy_input_test=noisy_input_test.astype('float32')
print(noisy_input_test.shape)

############Denoise 3D data############

print(np.shape(input_test))
print(type(noisy_input_test))
#residual=model.predict(noisy_input_test[94:95])
#model = generator_model()
#model.load_weights('/content/drive/My Drive/VAE/models/HSI/gan3.h5')

In [ ]:
custom_objects={'ssim_metric': ssim_metric,'psnr_met2':psnr_met2,'ssim_loss':ssim_loss,'total_content_loss2d':total_content_loss2d,'ssim_metric2d':ssim_metric2d}
model1=load_model('models/HSI/EnsRL2D-G10I20_200_L2L1TV-spl-renorm.h5',custom_objects=custom_objects)
#model1=load_model('/content/drive/My Drive/VAE/models/HSI/EnsRL2D-G10I20_160_MSE-spl-renorm.h5',custom_objects=custom_objects)
#model2=load_model('/content/drive/My Drive/VAE/models/HSI/EnsRL2DG10I10_020_L1-SSIM.h5',custom_objects=custom_objects)
#model3=load_model('/content/drive/My Drive/VAE/models/HSI/EnsRL2DG10I10_030_L1-SSIM.h5',custom_objects=custom_objects)
#model4=load_model('/content/drive/My Drive/VAE/models/HSI/EnsRL2DG10I10_040_L1-SSIM.h5',custom_objects=custom_objects)
#model5=load_model('/content/drive/My Drive/VAE/models/HSI/EnsRL2DG10I10_050_L1-SSIM.h5',custom_objects=custom_objects)
#model6=load_model('/content/drive/My Drive/VAE/models/HSI/EnsRL2DG10I10_060_L1-SSIM.h5',custom_objects=custom_objects)
#model7=load_model('/content/drive/My Drive/VAE/models/HSI/EnsRL2DG10I10_070_L1-SSIM.h5',custom_objects=custom_objects)
#model8=load_model('/content/drive/My Drive/VAE/models/HSI/EnsRL2DG10I10_080_L1-SSIM.h5',custom_objects=custom_objects)
#model9=load_model('/content/drive/My Drive/VAE/models/HSI/EnsRL2DG10I10_090_L1-SSIM.h5',custom_objects=custom_objects)
#model10=load_model('/content/drive/My Drive/VAE/models/HSI/EnsRL2DG10I10_100_L1-SSIM.h5',custom_objects=custom_objects)
#model11=load_model('/content/drive/My Drive/VAE/models/HSI/EnsRL2DG10I10_110_L1-SSIM.h5',custom_objects=custom_objects)
#model12=load_model('/content/drive/My Drive/VAE/models/HSI/EnsRL2DG10I10_120_L1-SSIM.h5',custom_objects=custom_objects)
#model13=load_model('/content/drive/My Drive/VAE/models/HSI/EnsRL2DG10I10_130_L1-SSIM.h5',custom_objects=custom_objects)
#model14=load_model('/content/drive/My Drive/VAE/models/HSI/EnsRL2DG10I10_140_L1-SSIM.h5',custom_objects=custom_objects)
#model15=load_model('/content/drive/My Drive/VAE/models/HSI/EnsRL2DG10I10_150_L1-SSIM.h5',custom_objects=custom_objects)
#model16=load_model('/content/drive/My Drive/VAE/models/HSI/EnsRL2DG10I10_160_L1-SSIM.h5',custom_objects=custom_objects)
#model17=load_model('/content/drive/My Drive/VAE/models/HSI/EnsRL2DG10I10_170_L1-SSIM.h5',custom_objects=custom_objects)
#model18=load_model('/content/drive/My Drive/VAE/models/HSI/EnsRL2DG10I10_180_L1-SSIM.h5',custom_objects=custom_objects)
#model19=load_model('/content/drive/My Drive/VAE/models/HSI/EnsRL2DG10I10_190_L1-SSIM.h5',custom_objects=custom_objects)
#model20=load_model('/content/drive/My Drive/VAE/models/HSI/EnsRL2DG10I10_200_L1-SSIM.h5',custom_objects=custom_objects)

In [ ]:
#from scipy.io import loadmat
#img3d=loadmat('/content/drive/My Drive/VAE/data/HSI/wdc.mat')['img'];
#abc=np.array(img3d);
#print(img3d.shape)
#!pip install mat4py
#denoised1=model1.predict(np.stack((noisy_input_test[:,:,:,30,0:1],noisy_input_test[:,:,:,24,0:1],noisy_input_test[:,:,:,20,0:1]),3))
denoised=model1.predict(noisy_input_test[:,:,:,2])
#np.stack((out_np[30,:,:],out_np[20,:,:],out_np[10,:,:]),2)

In [ ]:
import time
denoised=np.empty((input_test.shape))
start_time = time.time()
for i in range(noisy_input_test.shape[3]):
  denoised[:,:,:,i,0:1]=model1.predict(noisy_input_test[:,:,:,i,0:1])
  #denoised1[:,:,:,i,0:1]=model1.predict(input_test[:,:,:,i,0:1])
print("--- %s seconds ---" % (time.time() - start_time))



In [ ]:
from skimage.color import rgb2gray
import matplotlib.pyplot as plt
from skimage.transform import rotate

fig, axes = plt.subplots(1, 3)
#fig, axes = plt.subplots(1, 2)
#axes[0].imshow(rgb2gray(input_test[0,:,:,20,0]),cmap=plt.cm.gray)
axes[0].imshow(np.stack((input_test[0,:,:,208,0],input_test[0,:,:,70,0],input_test[0,:,:,20,0]),2),cmap=plt.cm.gray)
#axes[0].imshow(np.concatenate(input_test[0,:,:,166,0],input_test[0,:,:,110,0],input_test[0,:,:,10,0]))
axes[0].set_title('Original image')
#axes[1].imshow(rgb2gray(noisy_input_test[0,:,:,20,0]),cmap=plt.cm.gray)
axes[1].imshow(np.stack((noisy_input_test[0,:,:,208,0],noisy_input_test[0,:,:,70,0],noisy_input_test[0,:,:,20,0]),2),cmap=plt.cm.gray)
axes[1].set_title('Noisy image')
#axes[2].imshow(denoised1[0,:,:,0],cmap=plt.cm.gray)
axes[2].imshow(np.stack((denoised[0,:,:,208,0],denoised[0,:,:,70,0],denoised1[0,:,:,20,0]),2))
axes[2].set_title('Denoised image')

In [ ]:
#print(np.shape(denoised1))
from scipy.io import savemat
savemat('/content/drive/My Drive/VAE/results/HSI/salinas-10-20L3.mat', {'denoised':denoised})
#savemat("den.mat", denoised1)

In [ ]:
from skimage.color import rgb2gray
import matplotlib.pyplot as plt
from skimage.transform import rotate

fig, axes = plt.subplots(3, 5)
#fig, axes = plt.subplots(1, 2)
axes[0,0].imshow(rotate(rgb2gray(input_test[0,:,:,0,0]),90,resize=True),cmap=plt.cm.gray)
axes[0,0].set_title('Original image')
axes[0,1].imshow(rotate(rgb2gray(noisy_input_test[0,:,:,0,0]),90,resize=True),cmap=plt.cm.gray)
axes[0,1].set_title('Noisy image')
axes[0,2].imshow(rotate(denoised1[0,:,:,0],90,resize=True),cmap=plt.cm.gray)
axes[0,2].set_title('Denoised image 1')
axes[0,3].imshow(rotate(denoised2[0,:,:,0],90,resize=True),cmap=plt.cm.gray)
axes[0,3].set_title('Denoised image 2')
#axes[0,4].imshow(rotate(denoised3[0,:,:,0],90,resize=True),cmap=plt.cm.gray)
#axes[0,4].set_title('Denoised image 3')
axes[1,0].imshow(rotate(denoised4[0,:,:,0],90,resize=True),cmap=plt.cm.gray)
axes[1,0].set_title('Denoised image 4')
#axes[1,1].imshow(rotate(denoised5[0,:,:,0],90,resize=True),cmap=plt.cm.gray)
#axes[1,1].set_title('Denoised image 5')
axes[1,2].imshow(rotate(denoised6[0,:,:,0],90,resize=True),cmap=plt.cm.gray)
axes[1,2].set_title('Denoised image 6')
axes[1,3].imshow(rotate(denoised7[0,:,:,0],90,resize=True),cmap=plt.cm.gray)
axes[1,3].set_title('Denoised image 7')
#axes[1,4].imshow(rotate(denoised8[0,:,:,0],90,resize=True),cmap=plt.cm.gray)
#axes[1,4].set_title('Denoised image 8')
#axes[2,0].imshow(rotate(denoised9[0,:,:,0],90,resize=True),cmap=plt.cm.gray)
#axes[2,0].set_title('Denoised image 9')
axes[2,1].imshow(rotate(denoised10[0,:,:,0],90,resize=True),cmap=plt.cm.gray)
axes[2,1].set_title('Denoised image 10')
plt.show()

In [ ]:
from PIL import Image
import matplotlib
import matplotlib.pyplot as plt
matplotlib.image.imsave('orig.png', input_test[0,:,:,0,0],cmap=plt.cm.gray)
matplotlib.image.imsave('noisy.png', noisy_input_test[0,:,:,0,0],cmap=plt.cm.gray)

In [ ]:
import matplotlib
matplotlib.image.imsave('prop.png', input_test[0,:,:,0,0],cmap=plt.cm.gray)